In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


In [ ]:
cd ice_sleep_detpj/

In [ ]:
cd data/

In [ ]:
!unzip -q /home/jovyan/ice_sleep_detpj/data/ice_control_dataset/ice_project/Validation/[원천]keypoint_준통제환경.zip -d /home/jovyan/ice_sleep_detpj/data/ice_control_dataset/ice_project/Validation/[원천]keypoint_준통제환경

In [ ]:
!pip install opencv-python

In [ ]:
!rm -rf /home/jovyan/ice_sleep_detpj/data/ice_lstm_seq_combined


In [ ]:
import json
import os

def convert_bbox(bbox, img_width, img_height):
    x_min, y_min, x_max, y_max = map(float, bbox)
    x_center = (x_min + x_max) / 2 / img_width
    y_center = (y_min + y_max) / 2 / img_height
    w = (x_max - x_min) / img_width
    h = (y_max - y_min) / img_height
    return x_center, y_center, w, h

def merge_bboxes(b1, b2):
    x_min = min(float(b1[0]), float(b2[0]))
    y_min = min(float(b1[1]), float(b2[1]))
    x_max = max(float(b1[2]), float(b2[2]))
    y_max = max(float(b1[3]), float(b2[3]))
    return [x_min, y_min, x_max, y_max]

def json_to_yolo(json_path, output_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    width = int(data['FileInfo']['Width'])
    height = int(data['FileInfo']['Height'])
    bboxes = data['ObjectInfo']['BoundingBox']

    result_lines = []

    # 👁️ 눈 (양쪽 눈 기준 합쳐서)
    leye = bboxes.get('Leye')
    reye = bboxes.get('Reye')
    if leye['isVisible'] and reye['isVisible']:
        is_open = leye['Opened'] or reye['Opened']
        class_id = 0 if is_open else 1  # 0: eye_open, 1: eye_closed
        merged_box = merge_bboxes(leye['Position'], reye['Position'])
        coords = convert_bbox(merged_box, width, height)
        result_lines.append(f"{class_id} {' '.join(map(str, coords))}")

    # 👄 입
    mouth = bboxes.get('Mouth')
    if mouth['isVisible']:
        class_id = 2 if mouth['Opened'] else 3  # 2: mouth_open, 3: mouth_closed
        coords = convert_bbox(mouth['Position'], width, height)
        result_lines.append(f"{class_id} {' '.join(map(str, coords))}")

    # 저장
    base_name = os.path.splitext(os.path.basename(json_path))[0]
    output_file = os.path.join(output_path, base_name + '.txt')
    with open(output_file, 'w') as f:
        f.write('\n'.join(result_lines))

# 📁 Jupyter 환경 기준 경로 (수정됨)
input_dir = 'data/ice_th/YOLO_dataset/labels/val'        # JSON 파일들
output_dir = 'data/ice_th/YOLO_dataset/labels/val_txt'   # YOLO txt 저장 경로

os.makedirs(output_dir, exist_ok=True)

# 변환 실행
for filename in os.listdir(input_dir):
    if filename.endswith('.json'):
        json_path = os.path.join(input_dir, filename)
        json_to_yolo(json_path, output_dir)

print("✅ 변환 완료!")


In [ ]:
import os
import numpy as np

def generate_y_true_from_folders(controlled_root, realroad_root, save_path):
    """
    통제환경은 졸음(1), 실제 도로 환경은 정상(0)으로 라벨링.
    폴더명(사람 단위)을 기준으로 라벨 생성 후 numpy 배열로 저장.

    Args:
        controlled_root (str): 통제환경 데이터 루트 폴더 경로
        realroad_root (str): 실제 도로 환경 데이터 루트 폴더 경로
        save_path (str): 저장할 y_true 경로 (npy 파일)

    Returns:
        y_true (np.ndarray): 라벨 배열 (0: 정상, 1: 졸음)
        person_ids (List[str]): 사람 ID 리스트 (폴더명 기준)
    """
    y_true = []
    person_ids = []

    # ✅ 통제환경: 졸음 = 1
    if os.path.exists(controlled_root):
        for person_folder in sorted(os.listdir(controlled_root)):
            full_path = os.path.join(controlled_root, person_folder)
            if os.path.isdir(full_path):
                person_ids.append(person_folder)
                y_true.append(1)

    # ✅ 실제 도로 환경: 정상 = 0
    if os.path.exists(realroad_root):
        for person_folder in sorted(os.listdir(realroad_root)):
            full_path = os.path.join(realroad_root, person_folder)
            if os.path.isdir(full_path):
                person_ids.append(person_folder)
                y_true.append(0)

    y_true = np.array(y_true)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    np.save(save_path, y_true)

    print(f"\n✅ y_true 저장 완료: {save_path}")
    print(f"   총 인원: {len(y_true)}명 | 졸음: {np.sum(y_true)}, 정상: {len(y_true) - np.sum(y_true)}")
    return y_true, person_ids


# ✅ Train 데이터 라벨링
controlled_root = "data/ice_control_dataset/ice_project/Training/[라벨]bbox_통제환경"
realroad_root = "data/ice_control_dataset/ice_project/Training/[라벨]bbox_실제도로환경/2.승용"
y_true_train, person_ids_train = generate_y_true_from_folders(
    controlled_root,
    realroad_root,
    save_path="data/y_true_labeling_train.npy"
)

# ✅ Validation 데이터 라벨링
controlled_val_root = "data/ice_control_dataset/ice_project/Validation/[라벨]bbox_통제환경"
realroad_val_root = "data/ice_control_dataset/ice_project/Validation/[라벨]bbox_실제도로환경/2.승용"
y_true_val, person_ids_val = generate_y_true_from_folders(
    controlled_val_root,
    realroad_val_root,
    save_path="data/y_true_labeling_val.npy"
)

# ✅ 전체 person_id와 y_true 합치기 (통합 label_dict 만들기 원할 경우)
all_person_ids = person_ids_train + person_ids_val
all_y_true = np.concatenate([y_true_train, y_true_val])

label_dict = dict(zip(all_person_ids, all_y_true))
print(f"\n✅ 통합 label_dict 생성 완료! 총 인원 수: {len(label_dict)}")


In [ ]:
import os
import json
import numpy as np

# ✅ Feature + person_id 추출 함수
def extract_feature_with_id(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    bb = data["ObjectInfo"]["BoundingBox"]

    def safe_area(pos):
        try:
            return max(0, (float(pos[2]) - float(pos[0])) * (float(pos[3]) - float(pos[1])))
        except:
            return 0.0

    left_eye_open = int(bb["Leye"]["Opened"])
    right_eye_open = int(bb["Reye"]["Opened"])
    mouth_open = int(bb["Mouth"]["Opened"])

    face_area = safe_area(bb["Face"]["Position"])
    left_eye_area = safe_area(bb["Leye"]["Position"])
    right_eye_area = safe_area(bb["Reye"]["Position"])
    mouth_area = safe_area(bb["Mouth"]["Position"])

    if face_area > 0:
        left_eye_ratio = left_eye_area / face_area
        right_eye_ratio = right_eye_area / face_area
        mouth_ratio = mouth_area / face_area
        face_area_log = np.log(face_area + 1)
    else:
        left_eye_ratio = right_eye_ratio = mouth_ratio = face_area_log = 0.0

    feature = [
        left_eye_open,
        right_eye_open,
        mouth_open,
        face_area_log,
        left_eye_ratio,
        right_eye_ratio,
        mouth_ratio,
    ]

    person_id = os.path.basename(os.path.dirname(json_path))
    return feature, person_id

# ✅ 시퀀스 생성 함수
def create_sequences_with_id(json_dir, label_dict, save_dir, seq_len=30):
    X_seq, y_true_seq = [], []
    person_to_features = {}

    for root, _, files in os.walk(json_dir):
        for jf in sorted(files):
            if jf.endswith('.json'):
                try:
                    fpath = os.path.join(root, jf)
                    feat, pid = extract_feature_with_id(fpath)
                    person_to_features.setdefault(pid, []).append(feat)
                except Exception as e:
                    print(f"⚠️ 오류: {jf} - {e}")

    for pid, features in person_to_features.items():
        for i in range(len(features) - seq_len):
            seq = features[i:i+seq_len]
            X_seq.append(seq)
            y_true_seq.append(label_dict.get(pid, 0))

    X_seq = np.array(X_seq)
    y_true_seq = np.array(y_true_seq)

    os.makedirs(save_dir, exist_ok=True)
    np.save(os.path.join(save_dir, "X_seq.npy"), X_seq)
    np.save(os.path.join(save_dir, "y_true_seq.npy"), y_true_seq)

    print(f"\n✅ 저장 완료: {save_dir}")
    print(f"  시퀀스 수: {len(X_seq)} | 졸음 비율: {np.mean(y_true_seq):.2f}")
    return X_seq, y_true_seq

# ✅ person_ids와 y_true 저장 로직
train_person_ids, val_person_ids = [], []
y_true_train, y_true_val = [], []

# 🔹 Train
train_paths = [
    ("data/ice_control_dataset/ice_project/Training/[라벨]bbox_통제환경", 1),
    ("data/ice_control_dataset/ice_project/Training/[라벨]bbox_실제도로환경/2.승용", 0)
]
for folder, label in train_paths:
    for person_folder in sorted(os.listdir(folder)):
        if os.path.isdir(os.path.join(folder, person_folder)):
            train_person_ids.append(person_folder)
            y_true_train.append(label)

# 🔹 Validation
val_paths = [
    ("data/ice_control_dataset/ice_project/Validation/[라벨]bbox_통제환경", 1),
    ("data/ice_control_dataset/ice_project/Validation/[라벨]bbox_실제도로환경/2.승용", 0)
]
for folder, label in val_paths:
    for person_folder in sorted(os.listdir(folder)):
        if os.path.isdir(os.path.join(folder, person_folder)):
            val_person_ids.append(person_folder)
            y_true_val.append(label)

# ✅ 저장
os.makedirs("data", exist_ok=True)
np.save("data/y_true_labeling_train.npy", np.array(y_true_train))
np.save("data/y_true_labeling_val.npy", np.array(y_true_val))
with open("data/person_ids_train.txt", "w") as f:
    f.writelines("\n".join(train_person_ids))
with open("data/person_ids_val.txt", "w") as f:
    f.writelines("\n".join(val_person_ids))

# ✅ label_dict 구성 (train + val 전체 기준)
all_person_ids = train_person_ids + val_person_ids
all_y_true = np.concatenate([y_true_train, y_true_val])
label_dict = dict(zip(all_person_ids, all_y_true))
print(f"\n📌 label_dict 총 인원: {len(label_dict)}")

# ✅ 시퀀스 생성
create_sequences_with_id(
    json_dir="data/ice_control_dataset/ice_project/Training/[라벨]bbox_통제환경",
    label_dict=label_dict,
    save_dir="data/lstm_seq_통제_ytrue/train"
)
create_sequences_with_id(
    json_dir="data/ice_control_dataset/ice_project/Validation/[라벨]bbox_통제환경",
    label_dict=label_dict,
    save_dir="data/lstm_seq_통제_ytrue/val"
)
create_sequences_with_id(
    json_dir="data/ice_control_dataset/ice_project/Training/[라벨]bbox_실제도로환경/2.승용",
    label_dict=label_dict,
    save_dir="data/lstm_seq_실제도로환경_ytrue/train"
)
create_sequences_with_id(
    json_dir="data/ice_control_dataset/ice_project/Validation/[라벨]bbox_실제도로환경/2.승용",
    label_dict=label_dict,
    save_dir="data/lstm_seq_실제도로환경_ytrue/val"
)


In [ ]:
import numpy as np
import os

# ✅ 실제 도로 환경 (승용)
X_train_s = np.load("data/lstm_seq_실제도로환경_ytrue/train/X_seq.npy")
y_train_s = np.load("data/lstm_seq_실제도로환경_ytrue/train/y_true_seq.npy")
X_val_s = np.load("data/lstm_seq_실제도로환경_ytrue/val/X_seq.npy")
y_val_s = np.load("data/lstm_seq_실제도로환경_ytrue/val/y_true_seq.npy")

# ✅ 통제 환경
X_train_c = np.load("data/lstm_seq_통제_ytrue/train/X_seq.npy")
y_train_c = np.load("data/lstm_seq_통제_ytrue/train/y_true_seq.npy")
X_val_c = np.load("data/lstm_seq_통제_ytrue/val/X_seq.npy")
y_val_c = np.load("data/lstm_seq_통제_ytrue/val/y_true_seq.npy")

# ✅ 통합
X_train_combined = np.concatenate([X_train_s, X_train_c], axis=0)
y_train_combined = np.concatenate([y_train_s, y_train_c], axis=0)
X_val_combined = np.concatenate([X_val_s, X_val_c], axis=0)
y_val_combined = np.concatenate([y_val_s, y_val_c], axis=0)

# ✅ 저장 디렉토리
save_dir = "data/ice_lstm_seq_combined_ytrue"
os.makedirs(save_dir, exist_ok=True)

# ✅ 저장
np.save(os.path.join(save_dir, "X_seq.npy"), X_train_combined)
np.save(os.path.join(save_dir, "y_seq.npy"), y_train_combined)
np.save(os.path.join(save_dir, "X_val_seq.npy"), X_val_combined)
np.save(os.path.join(save_dir, "y_val_seq.npy"), y_val_combined)

# ✅ 요약 출력
print("\n✅ y_true 기반 시퀀스 통합 저장 완료!")
print(f"🔹 Train: X = {X_train_combined.shape}, y = {y_train_combined.shape} (졸음: {np.sum(y_train_combined)}, 정상: {len(y_train_combined)-np.sum(y_train_combined)})")
print(f"🔹  Val : X = {X_val_combined.shape}, y = {y_val_combined.shape} (졸음: {np.sum(y_val_combined)}, 정상: {len(y_val_combined)-np.sum(y_val_combined)})")


In [ ]:
import os
import json
import numpy as np

# ✅ Feature + person_id + path 추출 함수
def extract_feature_with_id(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    bb = data["ObjectInfo"]["BoundingBox"]

    def safe_area(pos):
        try:
            return max(0, (float(pos[2]) - float(pos[0])) * (float(pos[3]) - float(pos[1])) )
        except:
            return 0.0

    left_eye_open = int(bb["Leye"]["Opened"])
    right_eye_open = int(bb["Reye"]["Opened"])
    mouth_open = int(bb["Mouth"]["Opened"])

    face_area = safe_area(bb["Face"]["Position"])
    left_eye_area = safe_area(bb["Leye"]["Position"])
    right_eye_area = safe_area(bb["Reye"]["Position"])
    mouth_area = safe_area(bb["Mouth"]["Position"])

    if face_area > 0:
        left_eye_ratio = left_eye_area / face_area
        right_eye_ratio = right_eye_area / face_area
        mouth_ratio = mouth_area / face_area
        face_area_log = np.log(face_area + 1)
    else:
        left_eye_ratio = right_eye_ratio = mouth_ratio = face_area_log = 0.0

    feature = [
        left_eye_open,
        right_eye_open,
        mouth_open,
        face_area_log,
        left_eye_ratio,
        right_eye_ratio,
        mouth_ratio,
    ]

    person_id = os.path.basename(os.path.dirname(json_path))
    return feature, person_id, json_path

# ✅ 커스텀 라벨 기반 시퀀스 생성 함수
def create_sequences_with_custom_label(json_dir, save_dir, seq_len=30):
    X_seq, y_seq = [], []
    person_to_features = {}

    for root, _, files in os.walk(json_dir):
        for jf in sorted(files):
            if jf.endswith('.json'):
                try:
                    fpath = os.path.join(root, jf)
                    feat, pid, full_path = extract_feature_with_id(fpath)
                    person_to_features.setdefault(pid, []).append((feat, full_path))
                except Exception as e:
                    print(f"⚠️ 오류: {jf} - {e}")

    for pid, feat_path_list in person_to_features.items():
        features, paths = zip(*feat_path_list)

        for i in range(len(features) - seq_len):
            seq_feats = features[i:i+seq_len]
            seq_paths = paths[i:i+seq_len]

            open_stat = np.array(seq_feats)[:, :2]
            both_closed = np.sum((open_stat[:, 0] == 0) & (open_stat[:, 1] == 0))

            joined_path_str = " ".join(seq_paths)
            if "졸음재현" in joined_path_str or "하품" in joined_path_str or both_closed >= 3:
                label = 1
            else:
                label = 0

            X_seq.append(seq_feats)
            y_seq.append(label)

    X_seq = np.array(X_seq)
    y_seq = np.array(y_seq)

    os.makedirs(save_dir, exist_ok=True)
    np.save(os.path.join(save_dir, "X_seq.npy"), X_seq)
    np.save(os.path.join(save_dir, "y_seq.npy"), y_seq)

    print(f"\n✅ 저장 완료: {save_dir}")
    print(f"  🔹 시퀀스 수: {len(X_seq)}")
    print(f"  👁 졸음 시퀀스 수: {np.sum(y_seq)} | 😀 정상 시퀀스 수: {len(y_seq) - np.sum(y_seq)}")
    return X_seq, y_seq

# ✅ 시퀀스 생성: train + val
X_train, y_train = create_sequences_with_custom_label(
    json_dir="data/ice_control_dataset/ice_project/Training/[라벨]bbox_통제환경",
    save_dir="data/lstm_seq_custom/train"
)

X_val, y_val = create_sequences_with_custom_label(
    json_dir="data/ice_control_dataset/ice_project/Validation/[라벨]bbox_통제환경",
    save_dir="data/lstm_seq_custom/val"
)

# ✅ 통합 저장
save_dir = "data/ice_lstm_seq_combined_custom"
os.makedirs(save_dir, exist_ok=True)
np.save(os.path.join(save_dir, "X_seq.npy"), X_train)
np.save(os.path.join(save_dir, "y_seq.npy"), y_train)
np.save(os.path.join(save_dir, "X_val_seq.npy"), X_val)
np.save(os.path.join(save_dir, "y_val_seq.npy"), y_val)

print("\n🎉 전체 통합 저장 완료!")
print(f"🔹 Train: X = {X_train.shape}, y = {y_train.shape} (졸음: {np.sum(y_train)}, 정상: {len(y_train)-np.sum(y_train)})")
print(f"🔹  Val : X = {X_val.shape}, y = {y_val.shape} (졸음: {np.sum(y_val)}, 정상: {len(y_val)-np.sum(y_val)})")


In [ ]:
import os
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall

# ✅ 1. 데이터 로드 (경로 수정)
X_train = np.load("data/ice_lstm_seq_combined_custom/X_seq.npy")
y_train = np.load("data/ice_lstm_seq_combined_custom/y_seq.npy")
X_val = np.load("data/ice_lstm_seq_combined_custom/X_val_seq.npy")
y_val = np.load("data/ice_lstm_seq_combined_custom/y_val_seq.npy")

print(f"\n✅ 데이터 로드 완료:")
print(f"🔹 X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"🔹 X_val:   {X_val.shape}, y_val:   {y_val.shape}")
print(f"📊 Train 클래스 분포: {dict(zip(*np.unique(y_train, return_counts=True)))}")
print(f"📊 Val   클래스 분포: {dict(zip(*np.unique(y_val, return_counts=True)))}")

# ✅ 2. Feature Scaling
scaler = StandardScaler()
X_train_2d = X_train.reshape(-1, X_train.shape[-1])
X_val_2d = X_val.reshape(-1, X_val.shape[-1])

X_train_scaled = scaler.fit_transform(X_train_2d).reshape(X_train.shape)
X_val_scaled = scaler.transform(X_val_2d).reshape(X_val.shape)

joblib.dump(scaler, "feature_scaler_custom.pkl")
print("✅ Feature Scaling 완료 및 저장됨: feature_scaler_custom.pkl")

# ✅ 3. 클래스 가중치 계산
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print(f"✅ 클래스 가중치: {class_weight_dict}")

# ✅ 4. 모델 정의
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), dropout=0.2, recurrent_dropout=0.2),
    BatchNormalization(),

    LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    BatchNormalization(),

    LSTM(32, dropout=0.2, recurrent_dropout=0.2),
    BatchNormalization(),

    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)
model.summary()

# ✅ 5. 콜백 정의
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
    ModelCheckpoint("best_model_lstm_custom.h5", monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)
]

# ✅ 6. 학습
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=30,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=1
)

# ✅ 7. 결과 요약
print("\n🎉 학습 완료!")
print(f"🔹 Train Loss: {min(history.history['loss']):.4f}")
print(f"🔹 Val Loss:   {min(history.history['val_loss']):.4f}")
print(f"🔹 Val Accuracy: {max(history.history['val_accuracy']):.4f}")
print(f"🔹 Val Precision: {max(history.history['val_precision']):.4f}")
print(f"🔹 Val Recall:    {max(history.history['val_recall']):.4f}")

# ✅ 8. 학습 곡선 시각화
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['precision'], label='Train Precision')
plt.plot(history.history['val_precision'], label='Val Precision')
plt.plot(history.history['recall'], label='Train Recall')
plt.plot(history.history['val_recall'], label='Val Recall')
plt.title('Precision/Recall Curve')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
!pip install imbalanced-learn

In [ ]:
# 클로드 코드 - 통제환경 + 실제도로 환경 결합 버전
import os
import json
import numpy as np
import joblib
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall

import os
import numpy as np
import matplotlib.pyplot as plt

# ✅ real_road 라벨 디버그용 리스트
realroad_debug_values = []

def extract_feature_with_id(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    bb = data['ObjectInfo']['BoundingBox']

    leye_open = int(bb['Leye']['Opened']) if 'Opened' in bb['Leye'] else 0
    reye_open = int(bb['Reye']['Opened']) if 'Opened' in bb['Reye'] else 0
    mouth_open = int(bb['Mouth']['Opened']) if 'Opened' in bb['Mouth'] else 0

    def safe_area(pos):
        try:
            return max(0, (float(pos[2]) - float(pos[0])) * (float(pos[3]) - float(pos[1])))
        except:
            return 0.0

    face_area = safe_area(bb['Face']['Position'])
    if face_area < 10:
        face_area = 1e-6

    left_eye_area = safe_area(bb['Leye']['Position'])
    right_eye_area = safe_area(bb['Reye']['Position'])
    mouth_area = safe_area(bb['Mouth']['Position'])

    left_eye_ratio = left_eye_area / face_area
    right_eye_ratio = right_eye_area / face_area
    mouth_ratio = mouth_area / face_area
    face_area_log = np.log(face_area + 1)

    features = [
        float(leye_open),
        float(reye_open),
        float(mouth_open),
        float(face_area_log),
        float(left_eye_ratio),
        float(right_eye_ratio),
        float(mouth_ratio)
    ]

    person_id = os.path.basename(json_path).split('_')[0]
    return features, person_id, json_path


# ✅ 시각화 함수
def visualize_realroad_debug(values):
    max_consecutives = [v['max_consecutive'] for v in values]
    closed_ratios = [v['closed_ratio'] for v in values]
    labels = [v['label'] for v in values]

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.hist([m for m, l in zip(max_consecutives, labels) if l == 1], bins=30, alpha=0.7, label='Drowsy')
    plt.hist([m for m, l in zip(max_consecutives, labels) if l == 0], bins=30, alpha=0.7, label='Normal')
    plt.title('Max Consecutive Eye Closures (real_road)')
    plt.xlabel('max_consecutive')
    plt.ylabel('Count')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.hist([r for r, l in zip(closed_ratios, labels) if l == 1], bins=30, alpha=0.7, label='Drowsy')
    plt.hist([r for r, l in zip(closed_ratios, labels) if l == 0], bins=30, alpha=0.7, label='Normal')
    plt.title('Closed Ratio (real_road)')
    plt.xlabel('closed_ratio')
    plt.ylabel('Count')
    plt.legend()

    plt.tight_layout()
    plt.show()

def create_combined_sequences(data_paths, save_dir, seq_len=16):
    """
    여러 환경의 데이터를 결합하여 시퀀스 생성
    data_paths: dict형태로 {'environment_name': 'path'} 
    """
    X_seq, y_seq = [], []
    person_to_features = {}

    # 디버깅을 위한 카운터
    label_debug = {
        'drowsy_path': 0,
        'drowsy_eyes': 0,
        'normal': 0,
        'by_environment': {}
    }

    print(f"🔄 다음 환경들의 데이터를 결합합니다:")
    for env_name, env_path in data_paths.items():
        print(f"  📁 {env_name}: {env_path}")
        label_debug['by_environment'][env_name] = {'drowsy': 0, 'normal': 0}

    # 각 환경별로 데이터 수집
    for env_name, json_dir in data_paths.items():
        print(f"\n🔍 {env_name} 환경 처리 중...")
        env_person_features = {}

        if not os.path.exists(json_dir):
            print(f"⚠️ 경로가 존재하지 않습니다: {json_dir}")
            continue

        file_count = 0
        for root, _, files in os.walk(json_dir):
            for jf in sorted(files):
                if jf.endswith('.json'):
                    try:
                        fpath = os.path.join(root, jf)
                        feat, pid, full_path = extract_feature_with_id(fpath)
                        if feat is not None:
                            combined_pid = f"{env_name}_{pid}"
                            env_person_features.setdefault(combined_pid, []).append((feat, full_path, env_name))
                            file_count += 1
                    except Exception as e:
                        print(f"⚠️ 오류: {jf} - {e}")

        print(f"  ✅ {file_count}개 파일 처리 완료")

        for pid, feat_list in env_person_features.items():
            person_to_features[pid] = feat_list

    print(f"\n📊 전체 수집된 사람 수: {len(person_to_features)}")

    # 시퀀스 생성
    for pid, feat_path_env_list in person_to_features.items():
        features, paths, envs = zip(*feat_path_env_list)

        if len(features) < seq_len:
            continue

        for i in range(len(features) - seq_len):
            seq_feats = features[i:i+seq_len]
            seq_paths = paths[i:i+seq_len]
            seq_env = envs[i]  # 첫 번째 프레임의 환경

            # ✅ 유효한 환경인지 확인
            assert seq_env in label_debug['by_environment'], f"❌ Unknown environment type: {seq_env}"

            open_stat = np.array(seq_feats)[:, :2]
            both_closed = (open_stat[:, 0] == 0) & (open_stat[:, 1] == 0)

            consecutive_closed = 0
            max_consecutive = 0
            for closed in both_closed:
                if closed:
                    consecutive_closed += 1
                    max_consecutive = max(max_consecutive, consecutive_closed)
                else:
                    consecutive_closed = 0

            closed_ratio = np.sum(both_closed) / len(both_closed)

            joined_path_str = " ".join(seq_paths)
            drowsy_keywords = ["졸음재현", "하품", "졸음", "drowsy", "yawn", "sleepy", "tired"]
            normal_keywords = ["정상", "normal", "alert", "awake"]

            path_indicates_drowsy = any(keyword in joined_path_str.lower() for keyword in drowsy_keywords)
            path_indicates_normal = any(keyword in joined_path_str.lower() for keyword in normal_keywords)
            


            if seq_env == "controlled":
                is_drowsy = (max_consecutive >= 4 or closed_ratio >= 0.35)
                if not is_drowsy and path_indicates_drowsy:
                    is_drowsy = True
                if max_consecutive < 2 and closed_ratio < 0.15: #이거 manx_consecutive 2로 바꿔서 하 생각.
                    is_drowsy = False
            elif seq_env == "real_road":
                is_drowsy = False
                if max_consecutive >= 5 or closed_ratio >= 0.3:
                    is_drowsy = True
                elif max_consecutive < 3 and closed_ratio < 0.15:
                    is_drowsy = False

            if path_indicates_normal and not path_indicates_drowsy:
                is_drowsy = False

            label = 1 if is_drowsy else 0


            # # ✅ 환경별 기준 라벨링 - 파일명 무시하고 눈 상태 기반 판단만 사용
            # if seq_env == "controlled":
            #     is_drowsy = (max_consecutive >= 8 or closed_ratio >= 0.35)
            # elif seq_env == "real_road":
            #     is_drowsy = (max_consecutive >= 8 or closed_ratio >= 0.35)

            # label = 1 if is_drowsy else 0


            # 디버깅 카운터
            if max_consecutive >= 8 or closed_ratio >= 0.35:
                label_debug['drowsy_eyes'] += 1
            elif path_indicates_drowsy:
                label_debug['drowsy_path'] += 1
            else:
                label_debug['normal'] += 1

            # ✅ 환경별 카운터
            if label == 1:
                label_debug['by_environment'][seq_env]['drowsy'] += 1
            else:
                label_debug['by_environment'][seq_env]['normal'] += 1

            X_seq.append(seq_feats)
            y_seq.append(label)

    X_seq = np.array(X_seq)
    y_seq = np.array(y_seq)

    os.makedirs(save_dir, exist_ok=True)
    np.save(os.path.join(save_dir, "X_seq.npy"), X_seq)
    np.save(os.path.join(save_dir, "y_seq.npy"), y_seq)

    print(f"\n✅ 저장 완료: {save_dir}")
    print(f"  🔹 총 시퀀스 수: {len(X_seq)}")
    print(f"  👁 졸음 시퀀스: {np.sum(y_seq)} ({np.sum(y_seq)/len(y_seq)*100:.1f}%)")
    print(f"  😀 정상 시퀀스: {len(y_seq) - np.sum(y_seq)} ({(len(y_seq) - np.sum(y_seq))/len(y_seq)*100:.1f}%)")
    print(f"  🔍 라벨링 상세:")
    print(f"    - 경로 기반 졸음: {label_debug['drowsy_path']}")
    print(f"    - 눈 상태 기반 졸음: {label_debug['drowsy_eyes']}")
    print(f"    - 정상: {label_debug['normal']}")
    print(f"  📊 환경별 분포:")
    for env, counts in label_debug['by_environment'].items():
        total = counts['drowsy'] + counts['normal']
        if total > 0:
            print(f"    {env}: 졸음 {counts['drowsy']} ({counts['drowsy']/total*100:.1f}%), 정상 {counts['normal']} ({counts['normal']/total*100:.1f}%)")

    return X_seq, y_seq

# ✅ 사용 예시
data_paths = {
    "controlled": "data/ice_control_dataset/ice_project/Training/[라벨]bbox_통제환경",
    "real_road": "data/ice_control_dataset/ice_project/Training/[라벨]bbox_실제도로환경/2.승용"
}

X, y = create_combined_sequences(data_paths, save_dir="data/lstm_seq_combined")

# ✅ 시각화 실행
visualize_realroad_debug(realroad_debug_values)


# 🔥 수정된 특성 스케일링 함수 - 이진 특성은 그대로 두고 연속 특성만 스케일링
def improved_feature_scaling(X_train, X_val):
    """이진 특성과 연속 특성을 분리해서 스케일링"""
    
    # 특성 분리: [left_eye_open, right_eye_open, mouth_open, face_area_log, left_eye_ratio, right_eye_ratio, mouth_ratio]
    binary_features = [0, 1, 2]  # 눈, 입 열림 상태 (이진) - 스케일링 하지 않음
    continuous_features = [3, 4, 5, 6]  # 면적, 비율 (연속) - 스케일링 적용
    
    print(f"🔍 특성 분리: 이진 특성 {len(binary_features)}개, 연속 특성 {len(continuous_features)}개")
    
    # 복사본 생성
    X_train_scaled = X_train.copy().astype(np.float32)
    X_val_scaled = X_val.copy().astype(np.float32)
    
    # 🔥 연속 특성만 스케일링
    scaler = StandardScaler()
    
    # 3D -> 2D 변환하여 스케일링 (연속 특성만)
    train_continuous = X_train[:, :, continuous_features].reshape(-1, len(continuous_features))
    val_continuous = X_val[:, :, continuous_features].reshape(-1, len(continuous_features))
    
    print(f"🔍 스케일링 전 연속 특성 통계:")
    print(f"  훈련 데이터: mean={train_continuous.mean(axis=0)}")
    print(f"  훈련 데이터: std={train_continuous.std(axis=0)}")
    
    # 스케일링 적용
    train_continuous_scaled = scaler.fit_transform(train_continuous)
    val_continuous_scaled = scaler.transform(val_continuous)
    
    print(f"🔍 스케일링 후 연속 특성 통계:")
    print(f"  훈련 데이터: mean={train_continuous_scaled.mean(axis=0)}")
    print(f"  훈련 데이터: std={train_continuous_scaled.std(axis=0)}")
    
    # 다시 3D로 복원하여 연속 특성 부분만 업데이트
    X_train_scaled[:, :, continuous_features] = train_continuous_scaled.reshape(
        X_train.shape[0], X_train.shape[1], len(continuous_features)
    )
    X_val_scaled[:, :, continuous_features] = val_continuous_scaled.reshape(
        X_val.shape[0], X_val.shape[1], len(continuous_features)
    )
    
    # 🔥 이진 특성은 원본 그대로 유지 (0 또는 1)
    print(f"🔍 이진 특성 확인 (스케일링 안함):")
    print(f"  left_eye_open 범위: {X_train_scaled[:,:,0].min():.3f} ~ {X_train_scaled[:,:,0].max():.3f}")
    print(f"  right_eye_open 범위: {X_train_scaled[:,:,1].min():.3f} ~ {X_train_scaled[:,:,1].max():.3f}")
    print(f"  mouth_open 범위: {X_train_scaled[:,:,2].min():.3f} ~ {X_train_scaled[:,:,2].max():.3f}")
    
    # 스케일러에 메타데이터 추가
    scaler.binary_features_ = binary_features
    scaler.continuous_features_ = continuous_features
    
    return X_train_scaled, X_val_scaled, scaler

# ✅ 데이터 분석 함수
def analyze_data_distribution(X, y, title="Data Analysis"):
    print(f"\n📊 {title}")
    print(f"Shape: X={X.shape}, y={y.shape}")
    
    unique, counts = np.unique(y, return_counts=True)
    for u, c in zip(unique, counts):
        print(f"Class {u}: {c} samples ({c/len(y)*100:.1f}%)")
    
    # 특성별 통계
    feature_names = ['left_eye_open', 'right_eye_open', 'mouth_open', 'face_area_log', 'left_eye_ratio', 'right_eye_ratio', 'mouth_ratio']
    
    print("\n📈 특성별 평균값 (졸음 vs 정상):")
    for i, fname in enumerate(feature_names):
        if len(X[y==1]) > 0 and len(X[y==0]) > 0:
            drowsy_mean = X[y==1, :, i].mean()
            normal_mean = X[y==0, :, i].mean()
            print(f"{fname}: 졸음={drowsy_mean:.3f}, 정상={normal_mean:.3f}, 차이={abs(drowsy_mean-normal_mean):.3f}")
        else:
            print(f"{fname}: 한쪽 클래스가 없어서 비교 불가")

# ✅ 개선된 모델 생성
def create_improved_model(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape, dropout=0.3, recurrent_dropout=0.3),
        BatchNormalization(),
        
        LSTM(32, return_sequences=False, dropout=0.3, recurrent_dropout=0.3),
        BatchNormalization(),
        
        Dense(32, activation='relu'),
        Dropout(0.4),
        Dense(16, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
    )
    
    return model



# ✅ 메인 실행 코드
if __name__ == "__main__":
    
    # 🔥 여러 환경의 데이터 경로 설정
    train_data_paths = {
        "controlled": "data/ice_control_dataset/ice_project/Training/[라벨]bbox_통제환경",
        "real_road": "data/ice_control_dataset/ice_project/Training/[라벨]bbox_실제도로환경/2.승용"
        
    }
    
    val_data_paths = {
        "controlled": "data/ice_control_dataset/ice_project/Validation/[라벨]bbox_통제환경", 
        "real_road": "data/ice_control_dataset/ice_project/Validation/[라벨]bbox_실제도로환경/승용"
    }
    
    # 1. 결합된 시퀀스 생성
    print("🔄 통제환경 + 실제도로환경 데이터 결합하여 시퀀스 생성...")
    
    try:
        X_train, y_train = create_combined_sequences(
            data_paths=train_data_paths,
            save_dir="data/lstm_seq_combined/train"
        )
        
        X_val, y_val = create_combined_sequences(
            data_paths=val_data_paths,
            save_dir="data/lstm_seq_combined/val"
        )
    except Exception as e:
        print(f"⚠️ 실제 데이터 로드 실패: {e}")
        print("🔄 더미 데이터로 테스트...")
        
        # 더미 데이터 생성 (균형 잡힌 클래스)
        np.random.seed(42)
        n_samples = 1000
        seq_len = 30
        n_features = 7
        
        X_train = np.random.random((n_samples, seq_len, n_features))
        # 이진 특성은 0 또는 1로 설정
        X_train[:, :, :3] = np.random.randint(0, 2, (n_samples, seq_len, 3))
        # 균형 잡힌 라벨 생성 (70% 정상, 30% 졸음)
        y_train = np.random.choice([0, 1], n_samples, p=[0.7, 0.3])
        
        X_val = np.random.random((200, seq_len, n_features))
        X_val[:, :, :3] = np.random.randint(0, 2, (200, seq_len, 3))
        y_val = np.random.choice([0, 1], 200, p=[0.7, 0.3])
    
    # 2. 데이터 분석
    analyze_data_distribution(X_train, y_train, "Combined Training Data")
    analyze_data_distribution(X_val, y_val, "Combined Validation Data")
    
    # 3. 클래스 균형 체크
    class_ratio = np.sum(y_train) / len(y_train)
    print(f"\n🔍 전체 클래스 분포:")
    print(f"  정상(0): {len(y_train) - np.sum(y_train)} ({(1-class_ratio)*100:.1f}%)")
    print(f"  졸음(1): {np.sum(y_train)} ({class_ratio*100:.1f}%)")
    
    if len(np.unique(y_train)) < 2:
        print("❌ 한쪽 클래스만 존재합니다. 라벨링 기준을 재검토해주세요.")
        exit()
    
    # 4. 🔥 수정된 특성 스케일링
    print("\n🔄 수정된 특성 스케일링...")
    X_train_scaled, X_val_scaled, scaler = improved_feature_scaling(X_train, X_val)
    
    # 스케일러 저장
    joblib.dump(scaler, "feature_scaler_combined_0604.pkl")
    print("✅ 스케일러 저장 완료: feature_scaler_combined_0604.pkl")

    
    # 5. 클래스 불균형 처리 (필요시)
    if class_ratio < 0.2 or class_ratio > 0.8:
        print("⚠️ 클래스 불균형 감지. 샘플링 고려...")
        
        from imblearn.over_sampling import SMOTE
        from imblearn.under_sampling import RandomUnderSampler
        
        # LSTM용 데이터를 2D로 변환
        X_train_2d = X_train_scaled.reshape(X_train_scaled.shape[0], -1)
        
        # 언더샘플링 또는 오버샘플링 적용
        if class_ratio < 0.2:  # 졸음이 너무 적으면 오버샘플링
            sampler = SMOTE(sampling_strategy=0.3, random_state=42)
        else:  # 졸음이 너무 많으면 언더샘플링
            sampler = RandomUnderSampler(sampling_strategy=0.4, random_state=42)
        
        X_train_resampled, y_train_resampled = sampler.fit_resample(X_train_2d, y_train)
        
        # 다시 3D로 변환
        X_train_scaled = X_train_resampled.reshape(-1, X_train_scaled.shape[1], X_train_scaled.shape[2])
        y_train = y_train_resampled
        
        print(f"리샘플링 후: {Counter(y_train)}")
    
    # 6. 클래스 가중치 계산
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    class_weight_dict = dict(zip(np.unique(y_train), class_weights))
    print(f"✅ 클래스 가중치: {class_weight_dict}")
    
    # 7. 모델 생성 및 학습
    model = create_improved_model((X_train_scaled.shape[1], X_train_scaled.shape[2]))
    model.summary()
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
        ModelCheckpoint("best_model_lstm_combined_0604.h5", monitor='val_loss', save_best_only=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-7, verbose=1)
    ]
    
    history = model.fit(
        X_train_scaled, y_train,
        validation_data=(X_val_scaled, y_val),
        epochs=30,
        batch_size=32,
        class_weight=class_weight_dict,
        callbacks=callbacks,
        verbose=1
    )
    
    # 8. 결과 평가
    y_pred = (model.predict(X_val_scaled) > 0.5).astype(int)
    print("\n📊 분류 보고서:")
    print(classification_report(y_val, y_pred))
    
    print("\n📊 혼동 행렬:")
    print(confusion_matrix(y_val, y_pred))
    
    # 9. 학습 곡선 시각화
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Loss Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 3, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('Accuracy Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 3, 3)
    plt.plot(history.history['precision'], label='Train Precision')
    plt.plot(history.history['val_precision'], label='Val Precision')
    plt.plot(history.history['recall'], label='Train Recall')
    plt.plot(history.history['val_recall'], label='Val Recall')
    plt.title('Precision/Recall Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    

In [ ]:
!python train.py --img 640 --batch 16 --epochs 30 --data data/ice_texi.yaml --weights yolov5s.pt --name ice_texi_mouth --workers 8

In [ ]:
!python detect.py \
--weights runs/train/ice_eye_mouth20/weights/best.pt \
--source data/all_images_실도환_택시_val \
--img 640 \
--conf-thres 0.65 \
--iou-thres 0.45 \
--device 0 \
--save-txt \
--save-conf \
--project runs/detect \
--name ice_yolo_test20_실도환_택시_val \
--exist-ok
